In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

In [2]:
# --- Setup board ---
num_squares = 4  # 2x2 board
qc = QuantumCircuit(num_squares, num_squares)

# Put each square in equal superposition of mine/safe
for i in range(num_squares):
    qc.h(i)  # Hadamard → (|0> + |1>)/sqrt(2)

print("Initial quantum board: each square in |0> + |1> superposition")
print(qc)

Initial quantum board: each square in |0> + |1> superposition
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     └───┘
c: 4/═════
          
Checking square 0...
✅ Safe at square 0.
Checking square 2...
✅ Safe at square 2.
Checking square 1...
✅ Safe at square 1.


In [ ]:
# --- Game State ---
measured_squares = set()
simulator = Aer.get_backend('qasm_simulator')
shots = 1  # One shot = one game

def check_square(qc, square_idx):
    """Measure a square and collapse the wavefunction."""
    if square_idx in measured_squares:
        print(f"Square {square_idx} already revealed!")
        return qc

    print(f"Checking square {square_idx}...")
    qc.measure(square_idx, square_idx)  # Measure into matching classical bit

    # Simulate this measurement
    result = simulator.run(qc, shots=shots).result()
    counts = result.get_counts()
    outcome = list(counts.keys())[0]  # Single-shot outcome string

    # Qiskit returns bitstrings in reverse order (qubit 0 = rightmost)
    outcome_str = outcome[::-1]
    state = outcome_str[square_idx]

    if state == '1':
        print(f"💣 BOOM! Mine at square {square_idx}. Game over.")
    else:
        print(f"✅ Safe at square {square_idx}.")

    measured_squares.add(square_idx)
    return qc

In [ ]:
# --- Example Playthrough ---
qc = check_square(qc, 0)  # First move
qc = check_square(qc, 2)  # Second move
qc = check_square(qc, 1)  # Third move